In [ ]:
%pip install datasets

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Load your dataset
dataset = pd.read_csv("/content/drive/MyDrive/LAD/Curated-1000-IT-NL.tsv", sep="\t")

# Create a translation field
dataset['translation'] = dataset.apply(
    lambda row: {'it': row['italian'], 'nl': row['dutch']},
    axis=1
)

# Drop unnecessary columns
processed_dataset = dataset[['translation']]

# Split the dataset
train_data, val_data = train_test_split(processed_dataset, test_size=0.1, random_state=42)
train_data, test_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Convert to Hugging Face Dataset format
hf_train = Dataset.from_pandas(train_data)
hf_val = Dataset.from_pandas(val_data)
hf_test = Dataset.from_pandas(test_data)

# Combine into a DatasetDict
hf_dataset = DatasetDict({
    "train": hf_train,
    "validation": hf_val,
    "test": hf_test
})

# Save the dataset
hf_dataset.save_to_disk("/content/drive/MyDrive/LAD/Curated-1000-Splits")

# Verify the splits
print(hf_dataset)

Saving the dataset (0/1 shards):   0%|          | 0/810 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/90 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation', '__index_level_0__'],
        num_rows: 810
    })
    validation: Dataset({
        features: ['translation', '__index_level_0__'],
        num_rows: 100
    })
    test: Dataset({
        features: ['translation', '__index_level_0__'],
        num_rows: 90
    })
})


In [ ]:
%pip install evaluate
%pip install sacrebleu
%pip install rouge_score
%pip install bert_score
%pip install meteor
%pip install chrf

ERROR: Could not find a version that satisfies the requirement chrf (from versions: none)
ERROR: No matching distribution found for chrf


In [ ]:
import evaluate
import numpy as np

from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
from datasets import load_from_disk

# Load the pre-trained model and tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# Load the prepared dataset
dataset = load_from_disk("/content/drive/MyDrive/LAD/Curated-1000-Splits")
test_set = dataset['test']

# Load metrics
sacrebleu = evaluate.load("sacrebleu")
bertscore = evaluate.load("bertscore")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")
chfrc = evaluate.load("chrf")

# Set source and target language codes
tokenizer.src_lang = "it_IT"
target_lang_code = "nl_XX"

# Prepare lists for predictions and references
pre_tuning_predictions = []
references = []
sentences = []

# Translate the test set
for example in test_set:
    italian_text = example['translation']['it']
    dutch_reference = example['translation']['nl']

    # Tokenize the Italian input
    encoded_input = tokenizer(italian_text, return_tensors="pt")

    # Generate translation
    generated_tokens = model.generate(
        **encoded_input,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code]
    )

    # Decode the generated tokens
    dutch_translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    # Append to lists
    pre_tuning_predictions.append(dutch_translation)
    references.append(dutch_reference)
    sentences.append(italian_text)

# Compute metrics
sacrebleu_result = sacrebleu.compute(predictions=pre_tuning_predictions, references=references)
bertscore_result = bertscore.compute(predictions=pre_tuning_predictions, references=[ref[0] for ref in references], lang="nl")
rouge_result = rouge.compute(predictions=pre_tuning_predictions, references=[ref[0] for ref in references])
meteor_result = meteor.compute(predictions=pre_tuning_predictions, references=references)
chfrc_result = chfrc.compute(predictions=pre_tuning_predictions, references=references)

# Compute mean for BERTScore metrics
precision = np.mean(bertscore_result['precision'])
recall = np.mean(bertscore_result['recall'])
f1 = np.mean(bertscore_result['f1'])

# Print results
print(f"SacreBLEU: {sacrebleu_result['score']}")
print(f"BERTScore: Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
print(f"ROUGE: {rouge_result}")
print(f"METEOR: {meteor.compute(predictions=pre_tuning_predictions, references=references)}")
print(f"CHRF: {chfrc.compute(predictions=pre_tuning_predictions, references=references)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


SacreBLEU: 17.685943122753127
BERTScore: Precision: 0.6005, Recall: 0.6485, F1: 0.6231
ROUGE: {'rouge1': 0.0077777777777777776, 'rouge2': 0.0, 'rougeL': 0.0077777777777777776, 'rougeLsum': 0.0077777777777777776}
METEOR: {'meteor': 0.42165204274998674}
CHRF: {'score': 36.5644801413291, 'char_order': 6, 'word_order': 0, 'beta': 2}


In [ ]:
import evaluate
import numpy as np
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_from_disk

# Load the pre-trained model and tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# Load the prepared dataset
dataset = load_from_disk("/content/drive/MyDrive/LAD/Curated-1000-Splits")
train_set = dataset['train']
validation_set = dataset['validation']
test_set = dataset['test']

# Set source and target language codes
tokenizer.src_lang = "it_IT"
target_lang_code = "nl_XX"

# Tokenize the datasets
def preprocess_function(examples):
    # Access the 'translation' column which is now a list of dictionaries
    translations = examples['translation']

    # Extract Italian and Dutch texts for each example in the batch
    inputs = [translation['it'] for translation in translations]
    targets = [translation['nl'] for translation in translations]

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding=True)
    labels = tokenizer(targets, max_length=128, truncation=True, padding=True)

    # Assign labels to model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Preprocess datasets
train_set = train_set.map(preprocess_function, batched=True)
validation_set = validation_set.map(preprocess_function, batched=True)
test_set = test_set.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/LAD/mbart_fine_tuned",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    logging_dir="/content/drive/MyDrive/LAD/logs",
    logging_steps=500,
)

# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=validation_set,
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-8157716d4c98>:60: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fabiano-busca (fabiano-busca-vrije-universiteit-amsterdam). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,4.348882
2,No log,1.638004
3,No log,0.880715


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=153, training_loss=4.356851016773897, metrics={'train_runtime': 169.1777, 'train_samples_per_second': 14.364, 'train_steps_per_second': 0.904, 'total_flos': 339418234552320.0, 'train_loss': 4.356851016773897, 'epoch': 3.0})

In [ ]:
# Load metrics
sacrebleu = evaluate.load("sacrebleu")
bertscore = evaluate.load("bertscore")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")
chfrc = evaluate.load("chrf")

# Prepare lists for predictions and references
post_tuning_predictions = []
references = []

# Translate the test set
for example in test_set:
    italian_text = example['translation']['it']
    dutch_reference = example['translation']['nl']

    # Tokenize the Italian input
    encoded_input = tokenizer(italian_text, return_tensors="pt").to(model.device)

    # Generate translation
    generated_tokens = model.generate(
        **encoded_input,
        forced_bos_token_id=tokenizer.lang_code_to_id[target_lang_code]
    )

    # Decode the generated tokens
    dutch_translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

    # Append to lists
    post_tuning_predictions.append(dutch_translation)
    references.append([dutch_reference])

# Compute metrics
sacrebleu_result = sacrebleu.compute(predictions=post_tuning_predictions, references=references)
bertscore_result = bertscore.compute(predictions=post_tuning_predictions, references=[ref[0] for ref in references], lang="nl")
rouge_result = rouge.compute(predictions=post_tuning_predictions, references=[ref[0] for ref in references])
meteor_result = meteor.compute(predictions=post_tuning_predictions, references=references)
chfrc_result = chfrc.compute(predictions=post_tuning_predictions, references=references)

# Compute mean for BERTScore metrics
precision = np.mean(bertscore_result['precision'])
recall = np.mean(bertscore_result['recall'])
f1 = np.mean(bertscore_result['f1'])

# Print results
print(f"SacreBLEU: {sacrebleu_result['score']}")
print(f"BERTScore: Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
print(f"ROUGE: {rouge_result}")
print(f"METEOR: {meteor.compute(predictions=post_tuning_predictions, references=references)}")
print(f"CHRF: {chfrc.compute(predictions=post_tuning_predictions, references=references)}")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


SacreBLEU: 50.56906267139139
BERTScore: Precision: 0.9332, Recall: 0.9323, F1: 0.9326
ROUGE: {'rouge1': 0.7524617461425314, 'rouge2': 0.5709362702491354, 'rougeL': 0.7344519627210188, 'rougeLsum': 0.735872385587965}
METEOR: {'meteor': 0.7572200626253874}
CHRF: {'score': 70.80800961202962, 'char_order': 6, 'word_order': 0, 'beta': 2}


In [ ]:
# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Italian Sentence': sentences,
    'Dutch Reference': [ref[0] if isinstance(ref, list) else ref for ref in references],
    'Pre-Tuning Prediction': pre_tuning_predictions,
    'Post-Tuning Prediction': post_tuning_predictions
})

# Save the results as a TSV file
results_file_path = "/content/drive/MyDrive/LAD/test_results.tsv"
results_df.to_csv(results_file_path, sep="\t", index=False)

print(f"Results saved to {results_file_path}")

Results saved to /content/drive/MyDrive/LAD/test_results.tsv


In [ ]:
results_df

,Italian Sentence,Dutch Reference,Pre-Tuning Prediction,Post-Tuning Prediction
0,Voi lo conoscete Tom?,Kennen jullie Tom?,Do you know Tom?,Ken je Tom?
1,È la ragazza dei miei sogni.,Ze is het meisje van mijn dromen.,Het meisje van mijn dromen.,Ze is de meisje van mijn dromen.
2,Non sei per niente stupido.,Je bent helemaal niet dom.,Je bent helemaal niet stupid.,Je bent helemaal niet dom.
3,A Tom non piace per nulla il poker.,Tom houdt helemaal niet van poker.,Tom doesn't like poker at all.,Tom houdt helemaal niet van poker.
4,Guarda ancora.,Kijk nog eens.,Kijk nog eens.,Kijk nog eens.
...,...,...,...,...
85,Dopo la domenica viene il lunedì.,Na zondag komt maandag.,Na Sunday is Monday.,Na zondag komt maandag.
86,Siamo soddisfatti del risultato.,We zijn blij met het resultaat.,We're satisfied with the result.,We zijn tevreden met het resultaat.
87,Io amo la poesia.,Ik hou van poëzie.,Ik hou van poetry.,Ik hou van poëzie.
88,"Ciao, Tom.","Hallo, Tom.",Hallo Tom.,"Hallo, Tom."
